In [5]:
import sys
import os 
pwd = os.getcwd()
sys.path.append(pwd)

In [7]:
for p in sys.path:
    print(p)

c:\Users\Kirthan\AppData\Local\Programs\Python\Python312\python312.zip
c:\Users\Kirthan\AppData\Local\Programs\Python\Python312\DLLs
c:\Users\Kirthan\AppData\Local\Programs\Python\Python312\Lib
c:\Users\Kirthan\AppData\Local\Programs\Python\Python312

C:\Users\Kirthan\AppData\Roaming\Python\Python312\site-packages
C:\Users\Kirthan\AppData\Roaming\Python\Python312\site-packages\win32
C:\Users\Kirthan\AppData\Roaming\Python\Python312\site-packages\win32\lib
C:\Users\Kirthan\AppData\Roaming\Python\Python312\site-packages\Pythonwin
c:\Users\Kirthan\AppData\Local\Programs\Python\Python312\Lib\site-packages
c:\Users\Kirthan\OneDrive - Indian Institute of Technology Bombay\Desktop\Code tuts\Python\sequential networks


In [6]:
from Layers import Dense

ModuleNotFoundError: No module named 'layer'

In [ ]:
sys.path.remove(pwd)